# Optimización de hiperparámetros con Optuna



XGBoost

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE


from ydata_profiling import ProfileReport
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import optuna
import xgboost as xgb
from sklearn.metrics import f1_score
from sklearn.ensemble import VotingClassifier
from imblearn.over_sampling import SMOTE
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.preprocessing import OneHotEncoder
import pandas as pd


# Cargar los datos
df = pd.read_csv('stroke_dataset.csv')

# Preparación de los datos
df = pd.get_dummies(df, columns=['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status'])

# Selección de características
features = ['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi'] + \
           [col for col in df.columns if col.startswith(('gender_', 'ever_married_', 'work_type_', 'Residence_type_', 'smoking_status_'))]

X = df[features]
y = df['stroke']

# División de los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Escalado de características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [8]:
df_scaled_train = pd.DataFrame(X_train_scaled, columns=X_train.columns)


# Guardar como CSV
df_scaled_train.to_csv('df_scaled_train.csv', index=False)



In [11]:
from joblib import dump

dump(scaler, 'scalertotal.joblib')

['scalertotal.joblib']

In [5]:
important_features = [
    "age",
    "smoking_status_never smoked",
    "hypertension",
    "work_type_Private",
    "Residence_type_Rural",
    "heart_disease"
]

scaler_specific = StandardScaler()
scaler_specific.fit(df_train[important_features])

# Guardar el scaler específico
joblib.dump(scaler_specific, 'scaler_specific.joblib')

# Filtrar los conjuntos de datos
X_train_important = X_train[important_features]
X_test_important = X_test[important_features]

In [6]:

from xgboost import XGBClassifier

loaded_model = XGBClassifier()
loaded_model.load_model("xgboost_stroke_model_final.bin")

In [7]:
# Obtener las probabilidades de las predicciones
y_prob_xgb = loaded_model.predict_proba(X_test_important)[:, 1]

# Definir un nuevo umbral
new_threshold = 0.165  # Ajusta este valor según sea necesario

# Clasificar las predicciones basadas en el nuevo umbral
y_pred_loaded_model = (y_prob_xgb >= new_threshold).astype(int)

# Evaluar el rendimiento del modelo cargado
accuracy = accuracy_score(y_test, y_pred_loaded_model)
print(f"Accuracy del modelo cargado: {accuracy:.4f}")

# Imprimir el informe de clasificación
print("\nClassification Report del modelo cargado:")
print(classification_report(y_test, y_pred_loaded_model))



Accuracy del modelo cargado: 0.9137

Classification Report del modelo cargado:
              precision    recall  f1-score   support

           0       0.97      0.94      0.95       947
           1       0.26      0.40      0.32        50

    accuracy                           0.91       997
   macro avg       0.62      0.67      0.64       997
weighted avg       0.93      0.91      0.92       997



In [5]:
import xgboost as xgb
print(xgb.__version__)

1.6.2
